In [0]:
# setup spark on linux
from pyspark.sql import *

if SparkSession.getActiveSession() == None:
    print("Active spark session not found")
else:
    print("Active spark session found")

spark = (SparkSession.builder
         #.master("local") # local- no parallelizm at all, local[2] - 2 cores, local[*] - as many cores as local logical cores
         .appName("SparkSession#1")
         .enableHiveSupport() # enableHiveSupport() needed to make data persistent... 
         #.config("spark.driver.allowMultipleContexts", "true")
         .config("spark.sql.cbo.enabled", "true")
         .config("spark.sql.cbo.optimizer", "true")
         .getOrCreate())

print('spark version:', spark.version)
print('Done.')

Active spark session not found


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 11:20:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark version: 3.5.5
Done.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 45884)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/pyspark/accumulators.p

In [0]:
#print(list(locals().keys()))
#del cust
print(random.randint(0,1))

1


In [0]:
# prepare the data

import random
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

max_cust = 10
max_days=1000

# df_customer(id, name, nationality)
l_data = []
for c in range(max_cust):
    l_data.append([c, 'cust_' + f'{c}'.rjust(5,'0'), random.choice(['AT', 'CH', 'DE', 'HU'])])
        
df_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("nationality", StringType(), True),
])

df_customer = spark.createDataFrame(l_data, df_schema)
df_customer.show()

# df_payment(id, orig_cust_id, dest_cust_id, amount, currency, payment_timestamp)
l_data = []
for c in range(100):
    l_data.append([
        c, # id
        random.randint(0, max_cust-1),  # orig_cust_id
        random.randint(0, max_cust-1),  # dest_cust_id
        random.randint(1, 1000),  # amount
        random.choice(['EUR', 'CHF', 'HUF']), # currency
        datetime.strptime('20200101','%Y%m%d') + timedelta(days=random.randint(0, max_days)) if random.randint(0,1) == 1 else None, # payment_timestamp
    ])

df_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("orig_cust_id", IntegerType(), False),
    StructField("dest_cust_id", IntegerType(), False),
    StructField("amount", IntegerType(), False),
    StructField("currency", StringType(), False),
    StructField("payment_timestamp", TimestampType(), True),
])

df_payment = spark.createDataFrame(l_data, df_schema)
df_payment.show()

+---+----------+-----------+
| id|      name|nationality|
+---+----------+-----------+
|  0|cust_00000|         CH|
|  1|cust_00001|         CH|
|  2|cust_00002|         AT|
|  3|cust_00003|         DE|
|  4|cust_00004|         CH|
|  5|cust_00005|         HU|
|  6|cust_00006|         DE|
|  7|cust_00007|         CH|
|  8|cust_00008|         DE|
|  9|cust_00009|         HU|
+---+----------+-----------+

+---+------------+------------+------+--------+-------------------+
| id|orig_cust_id|dest_cust_id|amount|currency|  payment_timestamp|
+---+------------+------------+------+--------+-------------------+
|  0|           5|           3|   718|     CHF|2022-06-19 00:00:00|
|  1|           5|           3|   889|     CHF|2020-04-28 00:00:00|
|  2|           8|           4|   916|     CHF|               NULL|
|  3|           6|           6|   103|     HUF|               NULL|
|  4|           0|           9|   996|     CHF|               NULL|
|  5|           3|           7|   144|     HUF|  

In [0]:
# some transformation, normalization, standardization, DQ improvements
import pyspark.sql.functions as F

# add missing values
fixed_timestamp_str = "2019-01-01 00:00:00"
df_payment_filled = df_payment.withColumn('payment_timestamp_filled', 
                                          F
                                          .when(F.col('payment_timestamp').isNull(), F.lit(fixed_timestamp_str).cast("timestamp"))
                                          .otherwise(F.col('payment_timestamp'))
                                         )
df_payment_filled.show(10)
    
# add payment_month
df_payment_filled_ym=df_payment_filled.withColumn('payment_month', F.date_format(F.col('payment_timestamp_filled'), 'yyyyMM'))
df_payment_filled_ym.show(10)

+---+------------+------------+------+--------+-------------------+------------------------+
| id|orig_cust_id|dest_cust_id|amount|currency|  payment_timestamp|payment_timestamp_filled|
+---+------------+------------+------+--------+-------------------+------------------------+
|  0|           5|           3|   718|     CHF|2022-06-19 00:00:00|     2022-06-19 00:00:00|
|  1|           5|           3|   889|     CHF|2020-04-28 00:00:00|     2020-04-28 00:00:00|
|  2|           8|           4|   916|     CHF|               NULL|     2019-01-01 00:00:00|
|  3|           6|           6|   103|     HUF|               NULL|     2019-01-01 00:00:00|
|  4|           0|           9|   996|     CHF|               NULL|     2019-01-01 00:00:00|
|  5|           3|           7|   144|     HUF|               NULL|     2019-01-01 00:00:00|
|  6|           5|           2|   118|     CHF|               NULL|     2019-01-01 00:00:00|
|  7|           3|           4|   719|     CHF|               NULL|   

In [0]:
# q1: total monthly payment per customer (for each) which are CH nationals
import pyspark.sql.functions as F

# apply filter
df_customer_ch = df_customer.filter(F.col('nationality') == 'CH')

# join 
df_joined = df_customer_ch.alias('c').join(df_payment_filled_ym.alias('p'), on=F.col('c.id') == F.col('p.orig_cust_id'), how='left')
df_joined.show(10, truncate=False)

# aggregate, sum, order, round
df_result = (
        df_joined
        .groupBy(F.col('c.id'), F.col('p.payment_month'))
        .agg(F.round(F.sum(F.col('p.amount')),1).alias('sum_amount'))
        .orderBy(F.col('c.id'), F.col('p.payment_month'))
)
df_result.show(10, truncate=False)

+---+----------+-----------+---+------------+------------+------+--------+-------------------+------------------------+-------------+
|id |name      |nationality|id |orig_cust_id|dest_cust_id|amount|currency|payment_timestamp  |payment_timestamp_filled|payment_month|
+---+----------+-----------+---+------------+------------+------+--------+-------------------+------------------------+-------------+
|0  |cust_00000|CH         |94 |0           |5           |419   |CHF     |2021-10-21 00:00:00|2021-10-21 00:00:00     |202110       |
|0  |cust_00000|CH         |84 |0           |1           |842   |CHF     |2020-01-19 00:00:00|2020-01-19 00:00:00     |202001       |
|0  |cust_00000|CH         |77 |0           |4           |762   |EUR     |2020-12-21 00:00:00|2020-12-21 00:00:00     |202012       |
|0  |cust_00000|CH         |72 |0           |2           |357   |CHF     |NULL               |2019-01-01 00:00:00     |201901       |
|0  |cust_00000|CH         |57 |0           |7           |421 

In [0]:
# q2: average payment per orig/dest for customers which are CH nationals

import pyspark.sql.functions as F

df_customer_ch = df_customer.filter(F.col('nationality') == 'CH')

df_joined = (
        df_payment_filled_ym.alias('p')
        .join(df_customer_ch.alias('co'), on=F.col('co.id') == F.col('p.orig_cust_id'), how='inner')
        .join(df_customer_ch.alias('cd'), on=F.col('cd.id') == F.col('p.dest_cust_id'), how='inner')
)
df_joined.show(10, truncate=False)

df_result = (
        df_joined
        .groupBy(F.col('p.orig_cust_id'), F.col('p.dest_cust_id'))
        .agg(F.round(F.avg(F.col('p.amount')),1).alias('avg_amount'))
        .orderBy(F.col('p.orig_cust_id'), F.col('p.dest_cust_id'))
)
df_result.show(10, truncate=False)

+---+------------+------------+------+--------+-------------------+------------------------+-------------+---+----------+-----------+---+----------+-----------+
|id |orig_cust_id|dest_cust_id|amount|currency|payment_timestamp  |payment_timestamp_filled|payment_month|id |name      |nationality|id |name      |nationality|
+---+------------+------------+------+--------+-------------------+------------------------+-------------+---+----------+-----------+---+----------+-----------+
|10 |4           |1           |784   |HUF     |NULL               |2019-01-01 00:00:00     |201901       |4  |cust_00004|CH         |1  |cust_00001|CH         |
|13 |0           |4           |691   |CHF     |2020-05-06 00:00:00|2020-05-06 00:00:00     |202005       |0  |cust_00000|CH         |4  |cust_00004|CH         |
|14 |0           |7           |972   |CHF     |NULL               |2019-01-01 00:00:00     |201901       |0  |cust_00000|CH         |7  |cust_00007|CH         |
|33 |1           |7           |346

In [0]:
# Q3: achieve Q2 with Spark SQL not PySpark

# set up the temp views for the SQL
df_customer.createOrReplaceTempView('customer')
df_payment.createOrReplaceTempView('payment')

df_result_sql=spark.sql('''
WITH customer_ch as
(
  SELECT *
  FROM customer c
  WHERE c.nationality = 'CH'
)
SELECT orig_cust_id, dest_cust_id,
    round(avg(p.amount),1) as avg_amount
FROM payment p
    inner join
    customer_ch co
    on (p.orig_cust_id = co.id)
    inner join
    customer_ch cd
    on (p.dest_cust_id = cd.id)
GROUP BY orig_cust_id, dest_cust_id
ORDER BY orig_cust_id, dest_cust_id
''')

df_result_sql.show(10, truncate=False)
display(df_result_sql)

+------------+------------+----------+
|orig_cust_id|dest_cust_id|avg_amount|
+------------+------------+----------+
|0           |1           |512.0     |
|0           |4           |726.5     |
|0           |7           |486.3     |
|1           |1           |372.0     |
|1           |4           |614.0     |
|1           |7           |346.0     |
|4           |1           |806.0     |
|4           |7           |807.0     |
|7           |1           |192.5     |
|7           |4           |398.7     |
+------------+------------+----------+



orig_cust_id,dest_cust_id,avg_amount
0,1,512.0
0,4,726.5
0,7,486.3
1,1,372.0
1,4,614.0
1,7,346.0
4,1,806.0
4,7,807.0
7,1,192.5
7,4,398.7


In [0]:
# write out
df_result.write.format('parquet').mode('overwrite').save('my_data')
df_result.coalesce(1).write.format('csv').option('header', 'true').mode('overwrite').save('my_data1')